### Riddler Classic
From Dave Moran comes a pill-popping puzzle:

I’ve been prescribed to take 1.5 pills of a certain medication every day for 10 days, so I have a bottle with 15 pills. Each morning, I take two pills out of the bottle at random.

On the first morning, these are guaranteed to be two full pills. I consume one of them, split the other in half using a precision blade, consume half of that second pill, and place the remaining half back into the bottle.

On subsequent mornings when I take out two pills, there are three possibilities:

I get two full pills. As on the first morning, I split one and place the unused half back into the bottle.
I get one full pill and one half-pill, both of which I consume.
I get two half-pills. In this case, I take out another pill at random. If it’s a half-pill, then I consume all three halves. But if it’s a full pill, I split it and place the unused half back in the bottle.
Assume that each pill — whether it is a full pill or a half-pill — is equally likely to be taken out of the bottle.

On the 10th day, I again take out two pills and consume them. In a rush, I immediately throw the bottle in the trash before bothering to check whether I had just consumed full pills or half-pills. What’s the probability that I took the full dosage, meaning I don’t have to dig through the trash for a remaining half-pill?

### Things To Note:

- We are guaranteed to NOT exceed given we have 15 pills over 10 days and need 1.5 per day. I only note this so I can recognize that on the 10th day I only need to be concerned with being under or equal to 1.5 pills.


In [4]:
import numpy as np
from matplotlib import pyplot as plt

pills = np.ones(15)
total = 0
day = 0
while total < 15:
    
    # choose pills (indices) until you get 1.5
    step = 0
    day += 1
    
    if day == 10:
        p1, p2 = np.random.choice(pills, 2, replace = False)
        if p1 + p2 == 1.5:
            print("Success")
        else:
            print("Fail")
    
    while step < 1.5:
        p = np.random.choice(np.arange(pills.size), 1, replace = False)
        step += (pills[p])
        
        # remove
        pills = np.delete(pills, p)
    
    if step == 2:
        pills = np.append(pills, 0.5)
        
    print(pills)
 
    total += 1.5


[1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.5]
[1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.5 0.5]
[1.  1.  1.  1.  1.  1.  1.  1.  1.  0.5 0.5 0.5]
[1.  1.  1.  1.  1.  1.  1.  0.5 0.5 0.5 0.5]
[1.  1.  1.  1.  1.  1.  0.5 0.5 0.5]
[1.  1.  1.  1.  1.  0.5 0.5]
[1.  1.  1.  1.  0.5]
[1.  1.  0.5 0.5]
[1.  0.5]
Success
[]


### Now that it is working, let's make it a bit easier to read

Really need to speed this up, a bit stuck on how to run all sims as a single matrix, but likely a way I am missing here.

In [2]:
sims = 10000
proper = 0

def establish_sim():
    """Initial conditions for simulation"""
    return np.ones(15), 1

def day_ten(pill_array):
    """Check if last two pills chosen sum to 1.5"""
    p1, p2 = np.random.choice(pill_array, 2, replace = False)
    if p1 + p2 == 1.5:
        return 1
    else:
        return 0

def before_day_ten(pill_array, day, day_dose = 0):
    """Process for pills on day 1-9"""
    
    while day_dose < 1.5:
        p = np.random.choice(np.arange(pill_array.size), 1, replace = False)
        day_dose += (pill_array[p])
        pill_array = np.delete(pill_array, p)

    if day_dose == 2:
        pill_array = np.append(pill_array, 0.5)
    
    return pill_array, day + 1

In [3]:
sim_list = [100, 1_000, 10_000, 100_000, 500_000, 1_000_000]
for sims in sim_list:
    proper = 0
    for _ in range(sims):

        # Set up starting conditions
        pills, day = establish_sim()
        
        # Run Days 1-9
        while day < 10:
            pills, day = before_day_ten(pills, day)

        # Day 10: Results
        proper += day_ten(pills)

    print(f"Proper Dosage Rate for {sims} sims: {proper / sims :.3f}")

Proper Dosage Rate for 100 sims: 0.800
Proper Dosage Rate for 1000 sims: 0.807
Proper Dosage Rate for 10000 sims: 0.792
Proper Dosage Rate for 100000 sims: 0.789
Proper Dosage Rate for 500000 sims: 0.791
Proper Dosage Rate for 1000000 sims: 0.791


### A visual

We can see convergence towards out value.

In [ ]:
results = {}
for sims in range(1_000, 250_000, 1_000):
    proper = 0
    for _ in range(sims):

        # Set up starting conditions
        pills, day = establish_sim()
        
        # Run Days 1-9
        while day < 10:
            pills, day = before_day_ten(pills, day)

        # Day 10: Results
        proper += day_ten(pills)
    
    # Add to results
    results[sims] = proper

# Plot Solutions
final_solutions = [round(v/k,3) for k,v in results.items()]
plt.plot(results.keys(), final_solutions)
plt.xlabel("Sim Size")
plt.ylabel("Proper Dosage Probability")
plt.title("Simulating Proper Dosage In Two Pills");

### Analytical Solution: 

TODO

Day 1: 100% we grab 2 full, so 13 full + 1 half, 100%
Day 2: (13/14) * (12/13) -> grab 2 full -> 11 full + 2 h
       (13/14) * (1/13) -> grab 1 full, 1 half -> 12 full

However, this explodes each step so a smarter approach is needed.